# How to use Python MySQL bindings for WoS

In this we demonstrate:

    1) How to open a connection to the Web of Science (WoS) database
    2) How to instantiate a cursor to the database and issue SELECT queries
    3) Different strategies for fetching results from the queries

## Library imports

There are different Python MySQL binding libraries that you can use. Two examples are [MySQLdb](http://mysql-python.sourceforge.net/MySQLdb.html) and [PyMySQL](https://github.com/PyMySQL/PyMySQL).

In [ ]:
import pymysql as mysql
import os

In [ ]:
try:
    woshost = os.environ['woshost']
    wosuser = os.environ['wosuser']
    wospasswd = os.environ['wospasswd']
except KeyError:
    with open('./.wos_creds', 'r') as fp:
        woshost = fp.readline().strip()
        wosuser = fp.readline().strip()
        wospasswd = fp.readline().strip()

## Opening the connection

The connection object requires the following information:
    
   * __host__ : the hostname (either user-readable or IP address of the server hosting the database)
   * __port__ : MySQL servers usually listen on port 3306
   * __user__ : username for the server (if necessary)
   * __passwd__ : password for the server (if necessary)
   * __db__ : database to use (optional)

In [ ]:
conn = mysql.connect(host=woshost, 
               port=3306, 
               user=wosuser,
               passwd=wospasswd,
               db='wos2')

## Opening a Cursor and issuing Queries

The cursor object lets us directly interact with the data, through issuing queries and fetching data. There are two different cursors in pymysql that implement different strategies for how to store/deliver query results. 

** For more information check out the following documentation: [docs](http://pymysql.readthedocs.io/en/latest/modules/cursors.html)**

In [ ]:
num_results = 100

cur = conn.cursor()

cur.execute("SELECT wos_id FROM publications LIMIT {};".format(num_results))

print(cur.description)


## Retrieving Results

### Fetching all the results at once

In [ ]:
data = cur.fetchall()
for row in data:
    print(row[0])

### Iterating through the results


   **Important**: Once you fetch and iterate through the results, they are gone. It is expected that when you deal with the results, you save any of the information you need. 

To run the next cell and get results, you need to **rerun the query**. 

In [ ]:
data = cur.fetchone()
while data is not None:
    print(data[0])
    data = cur.fetchone() 

## Different SQL Strategies for Query Optimization

One of the great things about SQL (and in our case, MySQL) is the diversity of clauses for extracting information. By no means will this section be exhaustive, but we hope to provide some examples of different strategies that you can do to optimize your query.

Let's assume the usecase is that we have a set of document ids that we want to use to pull the rest of those documents' publication information.

In [ ]:
ids = [1, 2, 14000000, 15000000]

### 1. Search by comparison

In [ ]:
cur = conn.cursor()

query = "SELECT pubyear FROM publications WHERE id={};"

for id_ in ids:
    cur.execute(query.format(id_))
    data = cur.fetchall()
    print("id_: {}\t publication year: {}".format(id_, data[0]))

cur.close()

### 2. Search by membership

**N.B.** A caveat to using an approach like this is that you may be limited by the actual length of your query in bytes. If you get an error that suggests your query is too long, look for global variables in your server like `max_allowed_packet`. If you can configure that variable to lengthen the query length, then great. Otherwise, consider either splitting the query into different subqueries, or another approach entirely. 

In [ ]:
cur = conn.cursor()

query = "SELECT id, pubyear FROM publications WHERE id IN ("

for id_ in ids[:-1]:
    query += "{},".format(id_)
    
query += "{});".format(ids[-1])

cur.execute(query)
data = cur.fetchall()
for row in data:
    print("id: {}\t publication year: {}".format(row[0], row[1]))

cur.close()
    

In [ ]:
# Since the cursor works inside the connection, and you might want to 
# have another cursor sometime, you close the cursor first, and then 
# the connection

cur.close()
conn.close()